# Mount Your Container as a Drive with blobfuse

Rather than accessing Azure storage through API calls, it can be nice to have your Azure Blob Container mounted on your computer like a regular directory. There are two ways to do this:



## If you're using dask_cloudprovider

If you're running a `dask` cluster, then you can get your Blob Container auto-mounted on your computer without having to do the above. You can do this by registering your Blob Container from within your Workspace as a "Datastore". Then you can access it like a mounted drive from inside any of your VMs! (that's what the `datastores=ws.datastores.values()` line means in the `AzureMLCluster` call you saw). 

To do so, just go to your Workspace, and on the very bottom of the left hand menu click "Datastores." You should see two default Workspace datastores there, but let's make this interesting and add the file we just uploaded above. Now just click `+ Datastore` in the top left and connect it to the Blob Container you just created above, and you're done! Note you'll need your Storage Account key, which you can find by going back to the Azure Portal, clicking Storage Accounts, and clicking "Access Keys" on the left. 

Next time you spin up a `dask` cluster with `dask_cloudprovider`, you'll see your Container as a regular drive:

![azure_mounted_containers](images/azure_mounted_containers.png)



## If you're managing your own VM

One way to make your data available is to use `blobfuse` to mount your Blob Container as though it were just a drive on your AzureML machine. `blobfuse` is already installed on all the AzureML machines, so its relatively easy to mount a drive. ([here are the full docs](https://github.com/Azure/azure-storage-fuse)). Just open a terminal (either using SSH, or if you didn't enable SSH, use Jupyter Lab to open a terminal, and run the following after replacing the angle bracket labels with real values (you shouldn't have any angle brackets when you're done!):


```
# Create a cache on your local SSD
sudo mkdir /mnt/blobfusetmp -p
sudo chown azureuser /mnt/blobfusetmp

# Make the login details visible
export AZURE_STORAGE_ACCOUNT=<yourstorageaccountname>
export AZURE_STORAGE_ACCESS_KEY=<yourstorageaccountkey>

# Create a directory that
# will become the mounted drive

mkdir <path-to-where-you-want-to-mount>

blobfuse <path-to-where-you-want-to-mount> --container-name=<container-to-mount> --tmp-path=/mnt/blobfusetmp -o attr_timeout=240 -o entry_timeout=240 -o negative_timeout=120

```

And then your Blob container should appear in the folder you made with the `mkdir <path-to-where-you-want-to-mount>` argument. 